input data are two 8-band satellite scenes as TIFF files of the same region, at two different times
calculate PCA for both images and display them next to each other
subtract the first PCA band from time-1 image from the first PCA band from the time-2 image
display the difference raster

## Data Book

### Planet Labs

Band Map
1. coastal_blue  
2. blue  
3. green_i  
4. green  
5. yellow  
6. red  
7. rededge  
8. nir  

In [ ]:
import imageio.v3 as iio
import rasterio
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:

fp1 = '../../../data/frisco-change/frisco-22_psscene_analytic_8b_sr_udm2/files/PSScene/20220328_164657_27_2486/analytic_8b_sr_udm2/20220328_164657_27_2486_3B_AnalyticMS_SR_8b.tif'

fp2 = '../../../data/frisco-change/frisco-23_psscene_analytic_8b_sr_udm2/files/PSScene/20230211_164948_57_249a/analytic_8b_sr_udm2/20230211_164948_57_249a_3B_AnalyticMS_SR_8b.tif'


# Load the two satellite scenes as TIFF files
with rasterio.open(fp1) as src:
    scene1 = src.read()
    transform1 = src.transform

with rasterio.open(fp2) as src:
    scene2 = src.read()
    transform2 = src.transform


# Read the satellite image file into a numpy array
# scene1 = iio.imread(fp1)
# scene2 = iio.imread(fp1)

# # limit raster to AOI example
# import rasterio
# import geopandas as gpd
# from rasterio.mask import mask

# # Load the GeoJSON boundary using geopandas
# gdf = gpd.read_file('boundary.geojson')

# # Open the raster file and clip it to the boundary using the "mask" function
# with rasterio.open('image.tif') as src:
#     out_image, out_transform = mask(src, gdf.geometry, crop=True)

# # Update the metadata of the clipped image
# out_meta = src.meta.copy()
# out_meta.update({
#     "height": out_image.shape[1],
#     "width": out_image.shape[2],
#     "transform": out_transform
# })

# # Save the clipped image to disk
# with rasterio.open('clipped_image.tif', 'w', **out_meta) as dst:
#     dst.write(out_image)


In [ ]:
print(scene1.shape)
print(scene2.shape)

scene1_2d = scene1.reshape(-1, scene1.shape[0])
scene2_2d = scene2.reshape(-1, scene2.shape[0])

print(scene1_2d.shape)
print(scene2_2d.shape)

In [ ]:
# Calculate PCA for both images
pca1 = PCA(n_components=8)
pca1.fit(scene1_2d)

pca2 = PCA(n_components=8)
pca2.fit(scene2_2d)

In [ ]:
pca1_results = pca1.transform(scene1_2d)
pca2_results = pca2.transform(scene2_2d)


In [ ]:

print(pca1_results.shape)
print(pca2_results.shape)

In [ ]:
pca1_b1 = pca1_results[:, 0].reshape(scene1.shape[1], scene1.shape[2] )
pca2_b1 = pca2_results[:, 0].reshape(scene2.shape[1], scene2.shape[2] )

In [ ]:

# Display the first PCA band for both images
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(pca1_b1)
plt.title('PCA Band 1 - Scene 1')

plt.subplot(1, 2, 2)
plt.imshow(pca2_b1)
plt.title('PCA Band 1 - Scene 2')
plt.show()


In [ ]:

# Subtract the first PCA band from time-1 image from the first PCA band from the time-2 images
pca_diff = pca2.components_[0] - pca1.components_[0]

# Display the difference raster
plt.figure(figsize=(10, 5))
plt.imshow(pca_diff.reshape((src.height, src.width)), cmap='gray')
plt.title('Difference Raster')
plt.show()
